<a href="https://colab.research.google.com/github/1985shree/1985shree/blob/main/app_defect_validation_using_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Installing and importing modules

In [2]:
#  ! pip install spacy

In [4]:
#  ! python -m spacy download en_core_web_lg # download pretrained model

In [20]:
#  import nltk
#  nltk.download('stopwords')

In [79]:
# import regular modules
import pandas as pd
import numpy as np
import json
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer
import re
import spacy
nlp = spacy.load('en_core_web_lg')
spacy_tokenizer = Tokenizer(nlp.vocab)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer

nlp = spacy.load('en_core_web_lg')
spacy_tokenizer = Tokenizer(nlp.vocab)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer # to work in parallel on columns
from sklearn.base import BaseEstimator, TransformerMixin # to make fit_transform methods for free 
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# ! unzip -p Data.zip -d data

In [6]:
file_list = os.listdir()
file_names = [n for n in file_list if n[-5:] == '.json']
key_names = [n.replace('.json', "") for n in file_names]
# parsing json files:
data_list = []
for n in file_names:
    with open(n) as fi:
        d = json.load(fi)
        
        data_list.append(d)
        fi.close()

#data_list = [json.load(os.path.join(data_path, n)) for n in file_names]
data_dict = dict(zip(key_names, data_list))


In [7]:
df_cancelled = pd.DataFrame(data_dict['cancelled'])# packing into dataframes
df_noncancelled = pd.DataFrame(data_dict['noncancelled'])
df_sailorapp = pd.DataFrame(data_dict['sailorapp'])

In [8]:
df_cancelled.columns = [c.lower() for c in df_cancelled.columns]
df_noncancelled.columns = [c.lower() for c in df_noncancelled.columns]
df_sailorapp.columns = [c.lower() for c in df_sailorapp.columns]

In [9]:
# extracting details of the comments from json fomat data in commnets column
df_cancelled['cmt_details'] = df_cancelled.comments.apply(lambda x: [i['body'] for i in x])
df_cancelled['cmt_details'] = df_cancelled.cmt_details.apply(lambda x: "".join(x))

df_noncancelled['cmt_details'] = df_noncancelled.comments.apply(lambda x: [i['body'] for i in x])
df_noncancelled['cmt_details'] = df_noncancelled.cmt_details.apply(lambda x: "".join(x))

df_sailorapp['cmt_details'] = df_sailorapp.comments.apply(lambda x: [i['body'] for i in x])
df_sailorapp['cmt_details'] = df_sailorapp.cmt_details.apply(lambda x: "".join(x))

In [10]:
df_cancelled.cmt_details[1]

'Cancelling the defect because there was slight delay(expected) between ship to shore sync.  Hence the count difference.  Now the data has caught up and the counts are matching.'

In [11]:
df_cancelled['status'] = 0
df_noncancelled['status'] = 1 # setting the status target for each

In [12]:
# preparing df for train test split
df = pd.concat([df_cancelled, df_noncancelled]) # concatenate 2 dataframes into one

In [13]:
df.shape

(5689, 13)

In [14]:
df.head()

,status,description,comments,priority,rootcause,reporter,issuekey,summary,sprint,component,application,assigne,cmt_details
0,0,"Build: PI7.6 FE + 8.2 BE Hotfix , App Version:...",[],P3 - Medium,NA,Anusha Boosa,MSH-45790,CERT|SailorAPP| On-board | Cabin Services | I...,,NA,Sailor App,Anusha Boosa,
1,0,Ship parse count for housekeeping requests:\r\...,[{'author': {'accountId': '5e8b26768764110b816...,P3 - Medium,Code,Shanthi K,MSH-45793,Counts in the parse tables are not matching be...,PI8.4,NA,Crew VXP,Anagha Sebastian,Cancelling the defect because there was slight...
2,0,Hi [~accountid:5ade5093027e7a2ebfcbd97e] / [~a...,[{'author': {'accountId': '5ade5093027e7a2ebfc...,P2 - High,NA,Kapil Soni,MSH-45795,Magnolia | KAFKA | Ship Side Events are not co...,,NA,Sailor App,Kapil Soni,[~accountid:5d4c220bc54e920cf3b81002] - can yo...
3,0,Ship side team has successfully uploaded the s...,[{'author': {'accountId': '5caee6ee6c3b6b270f4...,P2 - High,NA,Exalate,MSH-45830,VV-Prod-Ship | VM - Bulk uploaded successfully...,,"""Crew VXP""",Crew VXP,Himanshu Kumawat,FYI: [DC Support|https://virginvoyages.atlassi...
4,0,Hit the application url and do the booking flo...,[{'author': {'accountId': '5dcdc9960e14000dd49...,P4 - Low,NA,Punithavalli,MSH-45840,DWP-QA-Spinner getting displayed in the top le...,,NA,eCommerce,Punithavalli,[~accountid:5d9b8c9033b8de0dc69a4f9a] I don’t ...


In [21]:
def clean_string(text,stem="None"):

    final_string = ""

    # make lower
    text = text.lower()
    
    # remove http
    text = re.sub(r"http.*\.[a-z]{2,3}","",text)

    # remove www
    text = re.sub(r"www.*\.[a-z]{2,3}","",text)
    
    # remove line breaks
    text = re.sub(r'\n','',text)

    # remove puncuation
    translator = str.maketrans('','',string.punctuation)
    text = text.translate(translator)

    # remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words.remove("don't")
    useless_words.remove('not')

    text_filtered = [word for word in text if not word in useless_words]

    # remove numbers
    text_filtered = [re.sub(r'\w*\d\w*','',w) for w in text_filtered]

    # stem or lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string
        
def apply_clean_string(df):
   arr = df.apply(lambda x: clean_string(str(x)))
   return pd.DataFrame(arr)  

In [106]:
countvectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
svc_classifier = LinearSVC()
vanilla_logistic = LogisticRegression()
tfidfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer, preprocessor = clean_string) # we will use both vectorizers
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')

forget pipelines let me try this first

In [ ]:
# df['description'] = df.description.apply(lambda x: clean_string(x))
# # df['comments'] = df.comments.apply(lambda x: clean_string(x))
# df['summary'] = df.summary.apply(lambda x: clean_string(x))
# # df['component'] = df.component.apply(lambda x: clean_string(x))
# df['cmt_details'] = df.cmt_details.apply(lambda x: clean_string(x))

In [17]:
df['soup'] = df['description'] + df['summary'] +df['application']+ df['component'] + df['cmt_details']

In [51]:
# checking individual features

In [133]:
df['summary'] = df.summary.apply(lambda x: clean_string(x)) # this gives a better score, models not so important, try features, categories too

In [23]:

count_pipe = Pipeline([
                 ('vectorizer', countvectorizer),

                 ('classifier', svc_classifier)])

In [ ]:
# count_pipe_logistic = Pipeline([
#                  ('vectorizer', countvectorizer),
#                  ('classifier', vanilla_logistic)])

In [24]:
tf_pipe = Pipeline([
                 ('vectorizer', tfidfvectorizer),
                 ('classifier', svc_classifier)])

In [25]:
X = df['summary']
y = df['status']

In [26]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=111)# test 

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=111)# train-val

In [28]:
count_pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 CountVectorizer(tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f4f2530ae50>)),
                ('classifier', LinearSVC())])

In [29]:
count_pipe_val = count_pipe.predict(X_val)
count_val_score = accuracy_score(count_pipe_val, y_val)


In [30]:
count_val_score

0.575192096597146

In [32]:
# count_pipe_logistic.fit(X_train, y_train)

In [33]:
# count_pipe_logistic_val = count_pipe_logistic.predict(X_val)
# count_logistic_val_score = accuracy_score(count_pipe_logistic_val, y_val)


In [34]:
# count_logistic_val_score

In [31]:
tf_pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(preprocessor=<function clean_string at 0x7f4f05c84820>,
                                 tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f4f2530ae50>)),
                ('classifier', LinearSVC())])

In [32]:
tf_pipe_val = tf_pipe.predict(X_val)
tf_val_score = accuracy_score(tf_pipe_val, y_val)

In [33]:
tf_val_score

0.575192096597146

In [135]:
# not using text data...just categorical data
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
xgb = XGBClassifier()
log_cat_pipe = Pipeline([
                 ('one_hot_encoder', ohe),

                 ('classifier', log_reg)])
xgb_cat_pipe = Pipeline([
                 ('one_hot_encoder', ohe),

                 ('classifier', xgb)])

In [136]:
X_cat = df[['priority', 'application']]
y_cat = df[['status']]

In [137]:
X_cat_train, X_cat_val, y_cat_train, y_cat_val = train_test_split(X_cat, y_cat, test_size=0.2, random_state=111)# train-val

In [74]:
# xgb_cat_pipe.fit(X_cat_train, y_cat_train)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('one_hot_encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('classifier', XGBClassifier())])

In [110]:
# xgb_cat_pipe

In [ ]:
# hu hu ha ha looks like 

In [45]:
# X_prior = df[['priority']]
# y_prior = df[['status']]

In [93]:
# X_prior_train, X_prior_val, y_prior_train, y_prior_val = train_test_split(X_prior, y_prior, test_size=0.2, random_state=111)# train-val

In [92]:
# cat_count_pipe.fit(X_prior_train, y_prior_train)

In [91]:
# cat_count_pipe_val = cat_count_pipe.predict(X_prior_val)
# cat_count_val_score = accuracy_score(cat_count_pipe_val, y_prior_val)

In [90]:
# cat_count_val_score

In [94]:
# X_app = df[['application']]
# y_app = df[['status']]

In [95]:
# X_app_train, X_app_val, y_app_train, y_app_val = train_test_split(X_app, y_app, test_size=0.2, random_state=111)# train-val

In [96]:
# cat_count_pipe.fit(X_app_train, y_app_train)

In [97]:
# cat_count_pipe_val = cat_count_pipe.predict(X_app_val)
# cat_count_val_score = accuracy_score(cat_count_pipe_val, y_app_val)

In [98]:
# cat_count_val_score

In [158]:
params_xgb = {'classifier__max_depth' : [3, 5, 7, 10, 20]},\
              {'classifier__min_child_weight': [1, 7, 20]},\
              {'classifier__gamma': [0.0005, 0.005, 0.1]}
xgb_cat_grid = GridSearchCV(xgb_cat_pipe, param_grid = params_xgb, cv=5,
                               n_jobs=-2, verbose=1)

In [129]:
xgb_cat_grid.fit(X_cat_train, y_cat_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('one_hot_encoder',
                                        OneHotEncoder(handle_unknown='ignore',
                                                      sparse=False)),
                                       ('classifier', XGBClassifier())]),
             n_jobs=-2,
             param_grid=({'classifier__max_depth': [3, 5, 7, 10, 20]},
                         {'classifier__min_child_weight': [1, 7, 20]},
                         {'classifier__gamma': [0.0005, 0.005, 0.1]}),
             verbose=1)

In [130]:
xgb_cat_grid.best_score_

0.8453094655070507

In [131]:
pd.DataFrame(xgb_cat_grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_depth,param_classifier__min_child_weight,param_classifier__gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.243286,0.004548,0.005984,0.000329,3,NaN,NaN,{'classifier__max_depth': 3},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
1,0.302958,0.013713,0.005759,0.000164,5,NaN,NaN,{'classifier__max_depth': 5},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
2,0.297013,0.010693,0.006087,0.000793,7,NaN,NaN,{'classifier__max_depth': 7},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
3,0.297233,0.009758,0.005895,0.000103,10,NaN,NaN,{'classifier__max_depth': 10},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
4,0.297279,0.012282,0.005958,0.001089,20,NaN,NaN,{'classifier__max_depth': 20},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
5,0.236664,0.002326,0.005392,0.000847,NaN,1,NaN,{'classifier__min_child_weight': 1},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
6,0.232270,0.008966,0.005541,0.000092,NaN,7,NaN,{'classifier__min_child_weight': 7},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
7,0.233662,0.007416,0.005503,0.000038,NaN,20,NaN,{'classifier__min_child_weight': 20},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
8,0.238244,0.005607,0.004827,0.000854,NaN,NaN,0.0005,{'classifier__gamma': 0.0005},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
9,0.238747,0.012383,0.004167,0.000382,NaN,NaN,0.005,{'classifier__gamma': 0.005},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1


In [101]:
xgb_cat_pipe.fit(X_cat_train, y_cat_train)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('one_hot_encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('classifier', XGBClassifier())])

In [102]:
xgb_cat_pipe_pred = xgb_cat_pipe.predict(X_cat_val)
xgb_cat_pipe_score = accuracy_score(xgb_cat_pipe_pred, y_cat_val)

In [103]:
xgb_cat_pipe_score

0.8506151142355008

In [154]:
params_log_cat = {'classifier__C' : [1, 3, 5, 10]}
                  
                  
# log_cat_grid = GridSearchCV(log_cat_pipe, param_grid = params_log_cat, cv=5,
#                                n_jobs=-2, verbose=1)

In [155]:
log_cat_grid = GridSearchCV(log_cat_pipe, param_grid = params_log_cat, cv=5,
                               n_jobs=-2, verbose=1)

In [156]:
log_cat_grid.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__one_hot_encoder', 'estimator__classifier', 'estimator__one_hot_encoder__categories', 'estimator__one_hot_encoder__drop', 'estimator__one_hot_encoder__dtype', 'estimator__one_hot_encoder__handle_unknown', 'estimator__one_hot_encoder__sparse', 'estimator__classifier__C', 'estimator__classifier__class_weight', 'estimator__classifier__dual', 'estimator__classifier__fit_intercept', 'estimator__classifier__intercept_scaling', 'estimator__classifier__l1_ratio', 'estimator__classifier__max_iter', 'estimator__classifier__multi_class', 'estimator__classifier__n_jobs', 'estimator__classifier__penalty', 'estimator__classifier__random_state', 'estimator__classifier__solver', 'estimator__classifier__tol', 'estimator__classifier__verbose', 'estimator__classifier__warm_start', 'estimator', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [157]:
log_cat_grid.fit(X_cat_train, y_cat_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

ValueError: ignored

In [143]:
log_cat_grid.best_score_

0.8453094655070507

In [144]:
pd.DataFrame(log_cat_grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.056003,0.006629,0.003622,0.000185,1,{'classifier__C': 1},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
1,0.060243,0.004804,0.003628,0.000157,3,{'classifier__C': 3},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
2,0.070594,0.011467,0.005603,0.002487,5,{'classifier__C': 5},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1
3,0.066368,0.006333,0.003623,0.000166,10,{'classifier__C': 10},0.841932,0.845055,0.856044,0.830769,0.852747,0.845309,0.008871,1


making a pipeline by plugging in the text cleaning step

In [42]:


naive_bayes = MultinomialNB()
knn = KNeighborsClassifier()
xgb = XGBClassifier(random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7)


# nb_pipe = Pipeline([('vectorizer', tfidfvectorizer, [''])
#                  ('classifier', svc_classifier)])
params = {'classifier__C': [1, 10, 100]}

knn_pipe = Pipeline([('vectorizer', tfidfvectorizer),
                 ('classifier', knn)])

xgb_pipe = Pipeline([('vectorizer', tfidfvectorizer),
                 ('classifier', xgb)])

params_knn = {'classifier__n_neighbors': [3, 5, 7, 10]},\
              {'classifier__leaf_size' : [20, 30, 40]}

params_xgb = {'classifier__max_depth' : [3, 5, 7, 10]}
        
         
tf_grid = GridSearchCV(tf_pipe, param_grid = params, cv=4,
                               n_jobs=-3, verbose=1)

# nb_grid = GridSearchCV(nb_pipe, param_grid = params, cv=4,
#                                n_jobs=-3, verbose=1)

knn_grid = GridSearchCV(knn_pipe, param_grid = params_knn, cv=4,
                               n_jobs=-3, verbose=1)


xgb_grid = GridSearchCV(xgb_pipe, param_grid = params_xgb, cv=4,
                               n_jobs=-3, verbose=1)



In [51]:
xgb_grid.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__vectorizer', 'estimator__classifier', 'estimator__vectorizer__analyzer', 'estimator__vectorizer__binary', 'estimator__vectorizer__decode_error', 'estimator__vectorizer__dtype', 'estimator__vectorizer__encoding', 'estimator__vectorizer__input', 'estimator__vectorizer__lowercase', 'estimator__vectorizer__max_df', 'estimator__vectorizer__max_features', 'estimator__vectorizer__min_df', 'estimator__vectorizer__ngram_range', 'estimator__vectorizer__norm', 'estimator__vectorizer__preprocessor', 'estimator__vectorizer__smooth_idf', 'estimator__vectorizer__stop_words', 'estimator__vectorizer__strip_accents', 'estimator__vectorizer__sublinear_tf', 'estimator__vectorizer__token_pattern', 'estimator__vectorizer__tokenizer', 'estimator__vectorizer__use_idf', 'estimator__vectorizer__vocabulary', 'estimator__classifier__base_score', 'estimator__classifier__booster', 'estimator__classifier__colsa

In [52]:
xgb_grid.fit(X_train_val, y_train_val)

Fitting 4 folds for each of 4 candidates, totalling 16 fits


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(preprocessor=<function clean_string at 0x7f31d78efdc0>,
                                                        tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f30ed7958b0>)),
                                       ('classifier',
                                        XGBClassifier(colsample_bytree=0.6,
                                                      random_state=42, seed=2,
                                                      subsample=0.7))]),
             n_jobs=-3, param_grid={'classifier__max_depth': [3, 5, 7, 10]},
             verbose=1)

In [53]:
xgb_grid.best_estimator_

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(preprocessor=<function clean_string at 0x7f31d78efdc0>,
                                 tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f30ae729940>)),
                ('classifier',
                 XGBClassifier(colsample_bytree=0.6, random_state=42, seed=2,
                               subsample=0.7))])

In [54]:
xgb_grid.best_score_

0.5706438489349304

In [58]:
xgb_grid.cv_results_['mean_test_score']

array([0.57064385, 0.57064385, 0.57064385, 0.57064385])

In [ ]:
df.priority.value_counts() # use cat encoder..one hot?

P2 - High      2431
P3 - Medium    1841
P1 - Urgent    1157
P4 - Low        260
Name: priority, dtype: int64

In [ ]:
df.rootcause.value_counts() # use cat encoder...label?

NA                     3952
Code                    950
Environmental           164
Data                    155
Content                 135
Design                  105
Invalid Issue           101
Requirements             60
Database                 45
User Error               11
Product Limitations       6
Hardware                  5
Name: rootcause, dtype: int64